# Whole notebook in a nutshell

bash /scratch/iss_decoding/nf/workflow-registration/local.sh \
    /scratch/tl10/registration/registration.yaml

In [1]:
! cat /scratch/tl10/regsistration/registration.yaml

feature_reg_yaml : /scratch/iss_decoding/data/registration/in/feat_reg.yaml
optflow_reg_yaml : /scratch/iss_decoding/data/registration/in/optflow_reg.yaml
sif_folder : /scratch/iss_decoding/sifs/
out_dir : ./out/


In [1]:
! cat /scratch/iss_decoding/data/registration/in/feat_reg.yaml

# Input
# If your input is not structured as a multichannel stack, you can specify
# individual channels per cycle like described bellow.
# If file contains more than one page, they will be treated as z-planes.

Input:
    InputImagePaths:
        Cycle 1: /scratch/iss_decoding/data/registration/in/NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_Anchor_Background_Background_Background_Meas5_A02_F1T1_max.ome_31671_demo_anchor_32477_40268_36712_44382.ome.tif
        Cycle 2: /scratch/iss_decoding/data/registration/in/NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP1-Cy5_AP1-Cy3_AP1-AF488_AP1-Atto425_Meas9_A02_F1T1_max.ome_31576_demo_cycle1_32867_38083_37146_42065.ome.tif
        Cycle 3: /scratch/iss_decoding/data/registration/in/NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP2-Cy5_AP2-Cy3_AP2-AF488_AP2-Atto425_Meas11_A02_F1T1_max.ome_31670_demo_cycle2_33582_39786_37759_43528.ome.tif
        Cycle 4: /scratch/iss_decoding/data/registration/in/NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP3-Cy5_AP3-Cy3_AP3-AF488_AP3-Atto425_Me

In [2]:
! cat /scratch/iss_decoding/data/registration/in/optflow_reg.yaml

# Input
# If your input image is a stack that contains channels from all cycles

Input:
  InputImagePaths:
    CycleStack: demo_feature_reg_result_stack.tif
  ReferenceCycle: 1
  ReferenceChannel: DAPI

# Output
# Images will be saved to a directory
#   /path/to/out/registered_imgs
# And because SaveOutputToCycleStack is false in this example,
# the file names will have the following patterns:
#   experiment_002_feature_reg_result_cyc001.tif
#   experiment_002_feature_reg_result_cyc00N.tif
#   experiment_002_optflow_reg_result_cyc001.tif
#   experiment_002_optflow_reg_result_cyc00N.tif

Output:
  OutputDir: ./
  OutputPrefix: demo_optflow_seg_
  SaveOutputToCycleStack: true

# Registration parameters
# Will do only non-linear optical flow based registration,
# will skip the feature based registration.
# For information on all the registration parameters check the file
# config_with_all_info.yaml

RegistrationParameters:
  OptFlowReg:
    NumberPyramidLevels: 3
    NumberIterationsPerLe

# Or step by step ...

In [1]:
import napari
from glob import glob
from dask_image.imread import imread

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

In [2]:
# Set the root directory
root = "/scratch/iss_decoding/workshop_data/registration/in/"

In [3]:
! ls {root}

NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP1-Cy5_AP1-Cy3_AP1-AF488_AP1-Atto425_Meas9_A02_F1T1_max.ome_31576_demo_cycle1_32867_38083_37146_42065.ome.tif
NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP2-Cy5_AP2-Cy3_AP2-AF488_AP2-Atto425_Meas11_A02_F1T1_max.ome_31670_demo_cycle2_33582_39786_37759_43528.ome.tif
NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP3-Cy5_AP3-Cy3_AP3-AF488_AP3-Atto425_Meas16_A02_F1T1_max.ome_31672_demo_cycle3_34215_39599_38506_43474.ome.tif
NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP4-Cy5_AP4-Cy3_AP4-AF488_AP4-Atto425_Meas18_A02_F1T1_max.ome_31668_demo_cycle4_34426_40028_38796_43910.ome.tif
NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP5-Cy5_AP5-Cy3_AP5-AF488_AP5-Atto425_Meas20_A02_F1T1_max.ome_31669_demo_cycle5_34889_39511_39138_43313.ome.tif
NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP6-Cy5_AP6-Cy3_AP6-AF488_AP6-Atto425_Meas23_A02_F1T1_max.ome_31673_demo_cycle6_33329_39612_37600_43175.ome.tif
NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_Anchor_Background_Background_Background_Meas5_A02_F1T1_max.ome_31671_demo_anc

In [4]:
# Load the reference and moving images
ref_cyc = imread(f"{root}/NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_Anchor_Background_Background_Background_Meas5_A02_F1T1_max.ome_31671_demo_anchor_32477_40268_36712_44382.ome.tif")
moving_cyc = imread(f"{root}/NT_FLW_KR0116_DAHN58.1d-S63_Nucleus_AP1-Cy5_AP1-Cy3_AP1-AF488_AP1-Atto425_Meas9_A02_F1T1_max.ome_31576_demo_cycle1_32867_38083_37146_42065.ome.tif")

In [10]:
viewer = napari.Viewer()

In [6]:
ref_cyc.shape, moving_cyc.shape

((5, 4114, 4234), (5, 3981, 4278))

In [7]:
min_x = min(ref_cyc.shape[1], moving_cyc.shape[1]) # the minimum Y size
min_y = min(ref_cyc.shape[2], moving_cyc.shape[2]) # the minimum X size

In [8]:
ref_DAPI = ref_cyc[0] # DAPI is the first channel
ref_anchor = ref_cyc[1] # Anchor is the second channel
moving_DAPI = moving_cyc[0] # DAPI is the first channel

In [11]:
viewer.add_image(ref_DAPI, blending="additive", colormap="cyan")
viewer.add_image(moving_DAPI, blending="additive", colormap="magenta")

<Image layer 'moving_DAPI' at 0x7fbca8bfca00>

In [12]:
# Show misaligned images
# plt.imshow(ref_DAPI, cmap='viridis', alpha=0.9, vmin=180, vmax=2000)
# plt.imshow(moving_DAPI, cmap='Reds', alpha=0.3, vmin=180, vmax=2000)
# plt.axis('off')

In [12]:
ref_DAPI = ref_DAPI[:min_x, :min_y] # crop the reference image
moving_DAPI = moving_DAPI[:min_x, :min_y] # crop the moving image

In [13]:
ref_DAPI.shape, moving_DAPI.shape

((3981, 4234), (3981, 4234))

In [14]:
viewer.add_image(ref_DAPI, name="cropped_ref_DAPI", blending="additive", colormap="cyan")
viewer.add_image(moving_DAPI, name="cropped_moving_DAPI", blending="additive", colormap="magenta")

<Image layer 'cropped_moving_DAPI' at 0x7fbb4df09c40>

In [15]:
# plt.imshow(ref_DAPI, cmap='viridis', alpha=0.5, vmin=180, vmax=2000)
# plt.imshow(moving_DAPI, cmap='Reds', alpha=0.5, vmin=180, vmax=2000)
# plt.axis('off')

Microaligner:
https://github.com/VasylVaskivskyi/microaligner

In [16]:
from microaligner import FeatureRegistrator, transform_img_with_tmat

In [ ]:
# Register the images using the FeatureRegistrator
freg = FeatureRegistrator()
freg.ref_img = ref_DAPI.compute()
freg.mov_img = moving_DAPI.compute()
transformation_matrix = freg.register()

img2_feature_reg_aligned = transform_img_with_tmat(moving_cyc[0].compute(), moving_cyc[0].shape, transformation_matrix)

In [19]:
# Apply the transformation matrix to the other channels in the moving cycle
another_feature_reg_aligned = transform_img_with_tmat(moving_cyc[1].compute(), moving_cyc[0].shape, transformation_matrix)

In [20]:
viewer.add_image(ref_DAPI, name="cropped_ref_DAPI", blending="additive", colormap="cyan")
viewer.add_image(img2_feature_reg_aligned, name="aligned_cropped_moving_DAPI", blending="additive", colormap="magenta")

<Image layer 'aligned_cropped_moving_DAPI' at 0x7f3f8823e4f0>

In [21]:
# plt.imshow(ref_DAPI, cmap='viridis', alpha=0.9, vmin=180, vmax=2000)
# plt.imshow(img2_feature_reg_aligned, cmap='Reds', alpha=0.3, vmin=180, vmax=2000)
# plt.axis('off')

In [22]:
ref_DAPI.shape, img2_feature_reg_aligned.shape

((3981, 4234), (3981, 4278))

# Optical flow registration

In [17]:
img2_feature_reg_aligned = img2_feature_reg_aligned[:min_x, :min_y] # crop the registered DAPI image
another_feature_reg_aligned = another_feature_reg_aligned[:min_x, :min_y] # crop the registered Anchor image

In [18]:
# Register image with optical flow algorithm
from microaligner import OptFlowRegistrator, Warper 
ofreg = OptFlowRegistrator()
ofreg.ref_img = ref_DAPI.compute()
ofreg.mov_img = img2_feature_reg_aligned
flow_map = ofreg.register()

warper = Warper()
warper.image = img2_feature_reg_aligned
warper.flow = flow_map
img2_optflow_reg_aligned = warper.warp()

Pyramid factor 16
    MI score after: 0.25038305489844503 | MI score before: 0.24698044056989826
    Better alignment than before
Pyramid factor 8
    MI score after: 0.3390639129017814 | MI score before: 0.3189287121233531
    Better alignment than before
Pyramid factor 4
    MI score after: 0.40731024296778384 | MI score before: 0.35210115392152697
    Better alignment than before
Pyramid factor 2
    MI score after: 0.3509688039190979 | MI score before: 0.28848665621514646
    Better alignment than before


In [23]:
flow_map.shape, img2_optflow_reg_aligned.shape

((3981, 4234, 2), (3981, 4234))

In [24]:
viewer.add_image(img2_optflow_reg_aligned, name="opeflow_aligned_cropped_moving_DAPI", blending="additive", colormap="magenta")

<Image layer 'opeflow_aligned_cropped_moving_DAPI' at 0x7f8d18bc6f40>

In [22]:
# plt.imshow(ref_DAPI, cmap='viridis', alpha=0.9, vmin=180, vmax=2000)
# plt.imshow(img2_optflow_reg_aligned, cmap='Reds', alpha=0.3, vmin=180, vmax=2000)
# plt.axis('off')

# Wrap another channel

In [25]:
warper = Warper()
warper.image = another_feature_reg_aligned
warper.flow = flow_map
another_optflow_reg_aligned = warper.warp()

In [26]:
viewer.add_image(ref_anchor, name="ref_anchor_spots", blending="additive", colormap="cyan")
viewer.add_image(another_optflow_reg_aligned, name="optflow_aligned_cropped_moving_spots", blending="additive", colormap="magenta")

<Image layer 'optflow_aligned_cropped_moving_spots' at 0x7f8d9f6f9130>

In [26]:
# plt.imshow(ref_anchor, cmap='viridis', alpha=0.9, vmin=180)
# plt.imshow(another_optflow_reg_aligned, cmap='Greys', alpha=0.3, vmin=180, vmax=2000)
# plt.axis('off')